In [16]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import shutil
dir_path = "../../disk/Data/"
ano = "results.csv"

In [3]:
annotations = pd.read_csv(dir_path+ano, delimiter='|')
annotations

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
...,...,...,...
158910,998845445.jpg,0,A man in shorts and a Hawaiian shirt leans ov...
158911,998845445.jpg,1,"A young man hanging over the side of a boat ,..."
158912,998845445.jpg,2,A man is leaning off of the side of a blue an...
158913,998845445.jpg,3,"A man riding a small boat in a harbor , with ..."


In [4]:
annotations[' comment']

0          Two young guys with shaggy hair look at their...
1          Two young , White males are outside near many...
2          Two men in green shirts are standing in a yard .
3              A man in a blue shirt standing in a garden .
4                   Two friends enjoy time spent together .
                                ...                        
158910     A man in shorts and a Hawaiian shirt leans ov...
158911     A young man hanging over the side of a boat ,...
158912     A man is leaning off of the side of a blue an...
158913     A man riding a small boat in a harbor , with ...
158914     A man on a moored blue and white boat with hi...
Name:  comment, Length: 158915, dtype: object

In [10]:
keyword = ['fire',
           'forest fire',
           'firefighter',
           'burst',
           'explode',
           'explosion',
           'blast',
           'detonation',
           'detonated',
           'burned',
           
           'earthquake',
           'earth tremor',
           'vibration in the earth',
           'landslip',
           'avalanche',
           'landslide',
           'collapse',
           'cave in',
           'collapsed',
           'drought',
           
           'tempest',
           'windstorm',
           'rainstorm',
           'whirlwind',
           'flood',
           'inundate',
           
           'tsunami',
           'tidal wave',
           'hail',
           'hailstones',
           
           'hurricane',
           'typhoon',
           'cyclone',
           'tornado',
           'dust devil',
           'twister',
           
           'thunder',
           'lightning',
           'thunderstorm',
           'thunderbolt',
           
           'foggy day',
           'in the fog',
           'dense fog',
           
           'snow storm',
           'heavy snowfall',
           'heavy snow',
        ]
exclude_keyword = ['blazer','horse','candle','fireplace','fire place','ski','board','says','oven','cook', 'old', 'living', 'plug', 'hyr', 'food', 'young','kitchen', 'extinguisher',
                  'fireworks','vintage','department', 'kid', 'boy', 'grill','fire station','fox','engine', 'hyd','fires']

In [15]:
find = annotations[annotations[' comment'].str.contains('dododo', na=False, case=False)]

for i in keyword:
    find2 = annotations[annotations[' comment'].str.contains(i,na=False, case=False)]
    find = pd.concat([find,find2], ignore_index=True)
    print(f'keyword : [{i}] // found: {len(find2)}')
print(f'final : {len(find)}')
## 결측지 유발 키워드 삭제
for i in exclude_keyword:
    origin_len = len(find)
    find = find[~find[' comment'].str.contains(i, na=False, case=False)]
    print(f'excluded keyword:[{i}] found: {origin_len - len(find)}')
print(f'final : {len(find)}')
## 중복 이미지 삭제 (한 이미지에 대한 여러 caption)
find.drop_duplicates(['image_name'], inplace=True ,ignore_index = True, keep = 'first')
print(f'droped duplicate images from multiple captions, After dropped: {len(find)}')
## 인덱스 정렬
find.reset_index(inplace=True, drop=True)
#find

keyword : [fire] // found: 1060
keyword : [forest fire] // found: 0
keyword : [firefighter] // found: 176
keyword : [burst] // found: 8
keyword : [explode] // found: 4
keyword : [explosion] // found: 7
keyword : [blast] // found: 6
keyword : [detonation] // found: 0
keyword : [detonated] // found: 0
keyword : [burned] // found: 6
keyword : [earthquake] // found: 2
keyword : [earth tremor] // found: 0
keyword : [vibration in the earth] // found: 0
keyword : [landslip] // found: 0
keyword : [avalanche] // found: 1
keyword : [landslide] // found: 1
keyword : [collapse] // found: 6
keyword : [cave in] // found: 0
keyword : [collapsed] // found: 5
keyword : [drought] // found: 1
keyword : [tempest] // found: 0
keyword : [windstorm] // found: 0
keyword : [rainstorm] // found: 13
keyword : [whirlwind] // found: 0
keyword : [flood] // found: 36
keyword : [inundate] // found: 0
keyword : [tsunami] // found: 0
keyword : [tidal wave] // found: 0
keyword : [hail] // found: 16
keyword : [hailstones

In [19]:
find.sort_values(by=['image_name'],ascending=True, inplace=True)
find.reset_index(drop=True, inplace=True)
tmp = find['image_name'].value_counts().index.sort_values()

In [ ]:
cnt = 4
for i,key in enumerate(tmp):
    if cnt == 4:
        print('', end='\n\n\n\n')
        print(f'{(i+1)/4}.')
        cnt = 0
    print(f"'{key}'", end=' ')
    cnt+=1

In [ ]:
## show img status
plt.figure(figsize=(16,500))
for i in range(len(find)):
    plt.subplot(100,4,i+1)
    if i % 4 == 0:
        plt.title(f"{(i+1)/4}")
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    k = find['image_name'][i]
    path = "../../disk/Data/flickr30k_images/" + k
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    plt.imshow(im_rgb, cmap=plt.cm.binary)
    plt.xlabel(find[' comment'][i][:30] + '\n' + find[' comment'][i][30:60] + '\n image_name : ' + str(find['image_name'][i]) , loc='left', fontsize=10)
plt.show()

In [1]:
disaster_dataset = ['2226280160.jpg', '2652155912.jpg', '2858793859.jpg', '2890731828.jpg', '3258472448.jpg',
                    '3338082262.jpg', '381052465.jpg', '4115261994.jpg', '4130537606.jpg', '4685179348.jpg',
                    '4685212004.jpg', '4718871145.jpg', '4764493697.jpg', '4797878234.jpg', '4922742734.jpg',
                    '4946977529.jpg', '5123250615.jpg', '7162932481.jpg', '7567712136.jpg', '89404014.jpg',
                    '236819162.jpg', '4478794627.jpg', '574274795.jpg',
                    '122156386.jpg', '167001809.jpg',
                    '206290645.jpg', '211981411.jpg', '2217244400.jpg', '3445296377.jpg', '3706019147.jpg',
                    '540791807.jpg'
                   ]
print(len(disaster_dataset))

31


In [9]:
annotations[annotations['image_name'] == '2226280160.jpg'][' comment'].value_counts().index

Index([' A firefighter on his ladder up against a telephone pole investigating a problem , with smoky clouds in the background .',
       ' In the midst of a fire , a worker is repairing telephone and electrical wires .',
       ' A fireman on a ladder fixing a telephone pole with flames in the background .',
       ' A fireman on top of a ladder in the dark .',
       ' A person is at the top of a ladder .'],
      dtype='object')

#### Caption extraction

In [13]:
captions = []
for img_name in disaster_dataset:
    captions_5 = annotations[annotations['image_name'] == img_name]
    found = captions_5[captions_5[' comment'].str.contains('dododo', case=False)]
    
    for i in keyword:
        found2 = captions_5[captions_5[' comment'].str.contains(i, case=False)]
        found = pd.concat([found,found2], ignore_index=True)
    
     ## 결측지 유발 키워드 삭제
    for i in exclude_keyword:
        origin_len = len(found)
        found = found[~found[' comment'].str.contains(i, case=False)]
    
    extracted = found[' comment'].value_counts().index
    
    for j in extracted:
        #print(j)
        if len(j) < 250:
            captions.append(j.rstrip('\n').replace(",", ""))
print(f'num of captions : {len(captions)}')

num of captions : 92


In [14]:
print(captions)

[' A firefighter on his ladder up against a telephone pole investigating a problem  with smoky clouds in the background .', ' In the midst of a fire  a worker is repairing telephone and electrical wires .', ' A fireman on a ladder fixing a telephone pole with flames in the background .', ' A fireman on top of a ladder in the dark .', ' a red car is leaping through a ball of fire .', ' A gas station and a car on fire .', ' A large explosion occured near a red car at a gas station .', ' A red car is caught in an explosion .', ' One person wearing a black hooded sweatshirt is running away from a fire while someone else is close by behind him .', ' Two men with rags over their face are running down a street with something on fire .', ' Two people running away from an unknown object on fire and a water sprayer .', ' Two men running down the road while a fire burns .', ' People are running from a fire .', ' Three firefighters duck from fire and smoke coming from a building .', ' The firefigh

#### 데이터 이동

In [25]:
dest = '../Data/disaster_images/flickr30k_disaster/'
path = '../../disk/Data/flickr30k_images/'
for img_name in disaster_dataset:
    from_path = path+img_name
    if os.path.isfile(from_path) == True:
        to_path = dest+img_name
        #print(to_path)
        shutil.move(from_path,to_path)
        print(os.path.isfile(from_path))


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [19]:
print(os.path.isfile('../../disk/Data/flickr30k_images/'+'2226280160.jpg'))

True
